In [ ]:
!pip install transformers datasets accelerate Sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_6

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Mengunggah file CSV Anda ke Google Colab
# Misalnya: dari panel sebelah kiri, Anda bisa mengunggah file 'train.csv'

# Memuat dataset menggunakan Pandas
df = pd.read_csv('/content/dataset_summarization.csv', sep=';')

# Menampilkan beberapa baris pertama untuk memastikan formatnya benar
print(df.head(1))

                     title                                               text  \
0  Pemilihan Presiden 2024  Pemilihan Presiden 2024 di Indonesia diperkira...   

                                             summary  
0  Pemilihan Presiden 2024 di Indonesia diprediks...  


In [ ]:
# Membagi dataset menjadi dataset pelatihan dan validasi
train_test_split = df.sample(frac=0.8, random_state=42)  # 80% untuk pelatihan
validation_split = df.drop(train_test_split.index)  # 20% untuk validasi

# Mengonversi DataFrame menjadi dataset Hugging Face
train_dataset = Dataset.from_pandas(train_test_split)
validation_dataset = Dataset.from_pandas(validation_split)

# Menggabungkan dataset menjadi DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset
})

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "cahya/bert2bert-indonesian-summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/999M [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    inputs = [ex for ex in examples['text']]  # Menggunakan kolom 'text' sebagai input
    targets = [ex for ex in examples['summary']]  # Menggunakan kolom 'summary' sebagai target
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=150, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/76 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/19 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
import matplotlib.pyplot as plt

# Define your model, tokenizer, and tokenized_datasets here

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=15,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Epoch,Training Loss,Validation Loss
1,No log,1.314422
2,No log,0.951539
3,No log,0.682980
4,No log,0.563226
5,No log,0.532838
6,No log,0.518194
7,No log,0.533033
8,No log,0.536469
9,No log,0.561261
10,No log,0.558799


TrainOutput(global_step=75, training_loss=0.3630747985839844, metrics={'train_runtime': 1284.0925, 'train_samples_per_second': 0.888, 'train_steps_per_second': 0.058, 'total_flos': 52719101952000.0, 'train_loss': 0.3630747985839844, 'epoch': 15.0})

In [ ]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.5550488829612732, 'eval_runtime': 5.1493, 'eval_samples_per_second': 3.69, 'eval_steps_per_second': 0.388, 'epoch': 15.0}


## Save the model offline

In [ ]:
# Save the model and tokenizer
# trainer.save_model("./fine-tuned-model")
# tokenizer.save_pretrained("./fine-tuned-model")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub("24PDInsight-TextSummarization")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 40, 'min_length': 20, 'early_stopping': True, 'num_beams': 10, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


model.safetensors:   0%|          | 0.00/999M [00:00<?, ?B/s]

events.out.tfevents.1717095430.71c6d083f2f9.158.0:   0%|          | 0.00/13.6k [00:00<?, ?B/s]

events.out.tfevents.1717096915.71c6d083f2f9.158.1:   0%|          | 0.00/354 [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/PaceKW/results/commit/0a565cba211ca06fbf20c4cc86b817cf822b64a1', commit_message='24PDInsight-TextSummarization', commit_description='', oid='0a565cba211ca06fbf20c4cc86b817cf822b64a1', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Import necessary libraries
from transformers import BertTokenizer, EncoderDecoderModel

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("PaceKW/24PDInsight-Summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

# Load the model
model = EncoderDecoderModel.from_pretrained("PaceKW/24PDInsight-Summarization")

def summarize_text(transcription):
    # Generate summary
    input_ids = tokenizer.encode(transcription, return_tensors='pt')
    summary_ids = model.generate(input_ids,
                                  min_length=20,
                                  max_length=80,
                                  num_beams=10,
                                  repetition_penalty=2.5,
                                  length_penalty=1.0,
                                  early_stopping=True,
                                  no_repeat_ngram_size=2,
                                  use_cache=True,
                                  do_sample=True,
                                  temperature=0.8,
                                  top_k=50,
                                  top_p=0.95)

    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary_text

In [ ]:
# Sample transcription text in Indonesian
sample_transcription = """
Pandemi COVID-19 telah berdampak besar pada perekonomian global. Banyak bisnis yang terpaksa tutup,
mengakibatkan pengangguran massal dan ketidakstabilan keuangan. Pemerintah di seluruh dunia telah menerapkan berbagai
tindakan untuk mengurangi dampaknya, seperti paket stimulus dan dukungan untuk usaha kecil. Meskipun upaya ini dilakukan,
jalan menuju pemulihan diperkirakan akan panjang dan menantang. Pandemi juga mempercepat adopsi teknologi digital,
karena kerja jarak jauh dan layanan online menjadi lebih umum. Ke depan, akan sangat penting untuk mengatasi ketimpangan
yang diperburuk oleh krisis ini dan memastikan pemulihan yang inklusif bagi semua.
"""

# Summarize the sample transcription
summary = summarize_text(sample_transcription)

# Print the original and summarized texts
print("Original Transcription:")
print(sample_transcription)
print("\nSummarized Text:")
print(summary)

Original Transcription:

Pandemi COVID-19 telah berdampak besar pada perekonomian global. Banyak bisnis yang terpaksa tutup, 
mengakibatkan pengangguran massal dan ketidakstabilan keuangan. Pemerintah di seluruh dunia telah menerapkan berbagai 
tindakan untuk mengurangi dampaknya, seperti paket stimulus dan dukungan untuk usaha kecil. Meskipun upaya ini dilakukan, 
jalan menuju pemulihan diperkirakan akan panjang dan menantang. Pandemi juga mempercepat adopsi teknologi digital, 
karena kerja jarak jauh dan layanan online menjadi lebih umum. Ke depan, akan sangat penting untuk mengatasi ketimpangan 
yang diperburuk oleh krisis ini dan memastikan pemulihan yang inklusif bagi semua.


Summarized Text:
pandemi pandemi covid - 19 berdampak besar pada perekonomian global. banyak bisnis yang terpaksa tutup, mengakibatkan pengangguran massal dan ketidakstabilan keuangan.
